In [2]:
%reload_ext autoreload
%autoreload 2

import os
import sys
import time
import cv2

from joblib import Parallel, delayed

sys.path.append(os.path.join(os.environ['REPO_DIR'], 'utilities'))
from utilities2015 import *

from matplotlib.path import Path
%matplotlib inline

import math
import random

Setting environment for Gordon


In [3]:
labels = ['BackG', '5N', '7n', '7N', '12N', 'Pn', 'VLL', 
          '6N', 'Amb', 'R', 'Tz', 'RtTg', 'LRt', 'LC', 'AP', 'sp5']

n_labels = len(labels)

labels_index = dict((j, i) for i, j in enumerate(labels))

labels_from_surround = dict( (l+'_surround', l) for l in labels[1:])

labels_surroundIncluded_list = labels[1:] + [l+'_surround' for l in labels[1:]]
labels_surroundIncluded = set(labels_surroundIncluded_list)

labels_surroundIncluded_index = dict((j, i) for i, j in enumerate(labels_surroundIncluded_list))


In [14]:
# Build training and testing data

sift_dir = '/oasis/projects/nsf/csd395/wel144/2016-brainstem/sift-jpeg'

def compute_intersection_kernel_oneJob(f, i, j1):
    dist = np.minimum(train_data_normalized[f][i], train_data_normalized[f][j1:]).sum(axis=1) 
    return dist

exprmt = 10
train_frac = [0.8, 0.6, 0.5, 0.3, 0.1] #Must be in descending order
n_sample = 500

for e in xrange(exprmt):
    
    t = time.time()
    
    print '\nExperiment #%d: '%e
    
    train_data = [[] for tf in train_frac]
    train_labels = [[] for tf in train_frac]
    train_fnames = [[] for tf in train_frac]
    
    test_data = []
    test_labels = []
    test_fnames = []

    for name in labels:

        hists0 = bp.unpack_ndarray_file(sift_dir + '/train/MD589_%(name)s_histograms_l0.bp' % {'name': name})
        files = bp.unpack_ndarray_file(sift_dir + '/train/MD589_%(name)s_fnames.bp' % {'name': name})

        n_total = hists0.shape[0]
        print (name, n_total),

        random_indices = np.random.choice(range(n_total), min(n_total, n_sample), replace=False)
        
        random_train = [None for tf in train_frac]
        n_train = [None for tf in train_frac]
        train_hists = [None for tf in train_frac]
        train_files = [None for tf in train_frac]

        frac = int(train_frac[0]*len(random_indices))
        random_train[0] = random_indices[:frac]
        random_test = random_indices[frac:]
        n_train[0] = len(random_train[0])
        n_test = len(random_test)
        
        for f in xrange(1, len(train_frac)):
            random_train[f] = np.random.choice(random_train[0], int(train_frac[f]/train_frac[0]*len(random_train[0])), replace=False)
            n_train[f] = len(random_train[f])
    
        for f in xrange(0, len(train_frac)):
            train_hists[f] = np.c_[hists0[random_train[f]]]
            train_files[f] = files[random_train[f]].reshape((n_train[f],))
            train_data[f].append(train_hists[f])
            train_labels[f].append(np.ones((n_train[f], )) * labels_index[name])
            train_fnames[f].append(train_files[f])
            print '\tTrain %d: '%int(train_frac[f]*100), n_train[f],
            
        test_hists =  np.c_[hists0[random_test]]
        test_files = files[random_test].reshape((n_test,))
        test_data.append(test_hists)
        test_labels.append(np.ones((n_test, )) * labels_index[name])
        test_fnames.append(test_files)
        print '\tTest: ', n_test

    for f in xrange(0, len(train_frac)):
        train_data[f] = np.concatenate(train_data[f])
        train_labels[f] = np.concatenate(train_labels[f])
        train_fnames[f] = np.concatenate(train_fnames[f])
        n_train[f] = train_data[f].shape[0]
        print train_data[f].shape
        print train_labels[f].shape
        print train_fnames[f].shape, train_fnames[f].dtype
        print train_fnames[f][0]
        
    test_data = np.concatenate(test_data)
    test_labels = np.concatenate(test_labels)
    test_fnames = np.concatenate(test_fnames)
    n_test = test_data.shape[0]
    print test_data.shape
    print test_labels.shape
    print test_fnames.shape, test_fnames.dtype
    print test_fnames[0]
    
    train_data_normalized = [None for tf in train_frac]
    train_dist_triangle = [None for tf in train_frac]
    train_dist_mat = [None for tf in train_frac]
    r = [None for tf in train_frac]
        
    for f in xrange(0, len(train_frac)):
        train_data_normalized[f] = train_data[f] / train_data[f].sum(axis=1)[:,None].astype(np.float)
        train_data_normalized[f] = np.nan_to_num(train_data_normalized[f])
        
        train_dist_triangle[f] = np.concatenate(Parallel(n_jobs=16)(delayed(compute_intersection_kernel_oneJob)(f, i, i) 
                                             for i in range(n_train[f])))
        
        train_dist_mat[f] = np.empty((n_train[f], n_train[f]))
        train_dist_mat[f][np.triu_indices(n_train[f])] = train_dist_triangle[f]
        r[f] = np.tril_indices(n_train[f])
        train_dist_mat[f][r[f]] = train_dist_mat[f].T[r[f]]
        print 'Train %d kernel shape: '%int(train_frac[f]*100), train_dist_mat[f].shape
        
    test_data_normalized = test_data / test_data.sum(axis=1)[:,None].astype(np.float)
    test_data_normalized = np.nan_to_num(test_data_normalized)
    
    for f in xrange(0, len(train_frac)):
        bp.pack_ndarray_file(train_data_normalized[f], sift_dir+'/preprocessed/train_level0_t%d_e%d_data.bp'%(int(train_frac[f]*100),e))
        bp.pack_ndarray_file(train_labels[f], sift_dir+'/preprocessed/train_level0_t%d_e%d_labels.bp'%(int(train_frac[f]*100),e))
        bp.pack_ndarray_file(train_fnames[f], sift_dir+'/preprocessed/train_level0_t%d_e%d_fnames.bp'%(int(train_frac[f]*100),e), chunk_size=255*4000)
        bp.pack_ndarray_file(train_dist_mat[f], sift_dir+'/preprocessed/train_level0_t%d_e%d_spm_dist_mat.bp'%(int(train_frac[f]*100),e))
                                            
    bp.pack_ndarray_file(test_data_normalized, sift_dir+'/preprocessed/test_level0_t%d_e%d_data.bp'%(100-int(train_frac[0]*100),e))
    bp.pack_ndarray_file(test_labels, sift_dir+'/preprocessed/test_level0_t%d_e%d_labels.bp'%(100-int(train_frac[0]*100),e))
    bp.pack_ndarray_file(test_fnames, sift_dir+'/preprocessed/test_level0_t%d_e%d_fnames.bp'%(100-int(train_frac[0]*100),e), chunk_size=255*4000)
    
    sys.stderr.write('Experiment #%d Time: %f seconds\n' % (e, (time.time()-t)))



Experiment #0: 
('BackG', 5105) 	Train 80:  400 	Train 60:  299 	Train 50:  250 	Train 30:  149 	Train 10:  50 	Test:  100
('5N', 1440) 	Train 80:  400 	Train 60:  299 	Train 50:  250 	Train 30:  149 	Train 10:  50 	Test:  100
('7n', 3444) 	Train 80:  400 	Train 60:  299 	Train 50:  250 	Train 30:  149 	Train 10:  50 	Test:  100
('7N', 2579) 	Train 80:  400 	Train 60:  299 	Train 50:  250 	Train 30:  149 	Train 10:  50 	Test:  100
('12N', 1230) 	Train 80:  400 	Train 60:  299 	Train 50:  250 	Train 30:  149 	Train 10:  50 	Test:  100
('Pn', 3042) 	Train 80:  400 	Train 60:  299 	Train 50:  250 	Train 30:  149 	Train 10:  50 	Test:  100
('VLL', 1287) 	Train 80:  400 	Train 60:  299 	Train 50:  250 	Train 30:  149 	Train 10:  50 	Test:  100
('6N', 154) 	Train 80:  123 	Train 60:  92 	Train 50:  76 	Train 30:  46 	Train 10:  15 	Test:  31
('Amb', 346) 	Train 80:  276 	Train 60:  206 	Train 50:  172 	Train 30:  103 	Train 10:  34 	Test:  70
('R', 1082) 	Train 80:  400 	Train 60:  299 	Tra

/oasis/projects/nsf/csd181/yuncong/virtualenv-1.9.1/yuncongve/lib/python2.7/site-packages/ipykernel/__main__.py:92: RuntimeWarning: invalid value encountered in divide


Train 80 kernel shape:  (5969, 5969)
Train 60 kernel shape:  (4462, 4462)
Train 50 kernel shape:  (3729, 3729)
Train 30 kernel shape:  (2224, 2224)


/oasis/projects/nsf/csd181/yuncong/virtualenv-1.9.1/yuncongve/lib/python2.7/site-packages/ipykernel/__main__.py:104: RuntimeWarning: invalid value encountered in divide
Experiment #0 Time: 13.070857 seconds


Train 10 kernel shape:  (745, 745)

Experiment #1: 
('BackG', 5105) 	Train 80:  400 	Train 60:  299 	Train 50:  250 	Train 30:  149 	Train 10:  50 	Test:  100
('5N', 1440) 	Train 80:  400 	Train 60:  299 	Train 50:  250 	Train 30:  149 	Train 10:  50 	Test:  100
('7n', 3444) 	Train 80:  400 	Train 60:  299 	Train 50:  250 	Train 30:  149 	Train 10:  50 	Test:  100
('7N', 2579) 	Train 80:  400 	Train 60:  299 	Train 50:  250 	Train 30:  149 	Train 10:  50 	Test:  100
('12N', 1230) 	Train 80:  400 	Train 60:  299 	Train 50:  250 	Train 30:  149 	Train 10:  50 	Test:  100
('Pn', 3042) 	Train 80:  400 	Train 60:  299 	Train 50:  250 	Train 30:  149 	Train 10:  50 	Test:  100
('VLL', 1287) 	Train 80:  400 	Train 60:  299 	Train 50:  250 	Train 30:  149 	Train 10:  50 	Test:  100
('6N', 154) 	Train 80:  123 	Train 60:  92 	Train 50:  76 	Train 30:  46 	Train 10:  15 	Test:  31
('Amb', 346) 	Train 80:  276 	Train 60:  206 	Train 50:  172 	Train 30:  103 	Train 10:  34 	Test:  70
('R', 1082) 	

...

Experiment #7 Time: 13.755141 seconds


Train 80 kernel shape:  (5969, 5969)
Train 60 kernel shape:  (4462, 4462)
Train 50 kernel shape:  (3729, 3729)
Train 30 kernel shape:  (2224, 2224)
Train 10 kernel shape:  (745, 745)

Experiment #9: 
('BackG', 5105) 	Train 80:  400 	Train 60:  299 	Train 50:  250 	Train 30:  149 	Train 10:  50 	Test:  100
('5N', 1440) 	Train 80:  400 	Train 60:  299 	Train 50:  250 	Train 30:  149 	Train 10:  50 	Test:  100
('7n', 3444) 	Train 80:  400 	Train 60:  299 	Train 50:  250 	Train 30:  149 	Train 10:  50 	Test:  100
('7N', 2579) 	Train 80:  400 	Train 60:  299 	Train 50:  250 	Train 30:  149 	Train 10:  50 	Test:  100
('12N', 1230) 	Train 80:  400 	Train 60:  299 	Train 50:  250 	Train 30:  149 	Train 10:  50 	Test:  100
('Pn', 3042) 	Train 80:  400 	Train 60:  299 	Train 50:  250 	Train 30:  149 	Train 10:  50 	Test:  100
('VLL', 1287) 	Train 80:  400 	Train 60:  299 	Train 50:  250 	Train 30:  149 	Train 10:  50 	Test:  100
('6N', 154) 	Train 80:  123 	Train 60:  92 	Train 50:  76 	Train 30

Experiment #8 Time: 13.011417 seconds


Train 80 kernel shape:  (5969, 5969)
Train 60 kernel shape:  (4462, 4462)
Train 50 kernel shape:  (3729, 3729)
Train 30 kernel shape:  (2224, 2224)
Train 10 kernel shape:  (745, 745)


Experiment #9 Time: 13.119460 seconds
